In [2]:
try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle

from kaveh.behavioral.oculomotor.session import session
from kaveh.toolbox import find_file
from neo.io import Spike2IO
from matplotlib import pyplot as plt
import numpy as np

import numpy as np
from matplotlib import pyplot as plt
# from neo.io import Spike2IO
from kaveh.toolbox import closest_argmin, find_file
from smr import File
from kaveh.plots import axvlines
import os
from kaveh.sorting.spikesorter import SimpleSpikeSorter
import sys

from neo import Spike2IO
from sklearn.mixture import GaussianMixture
import csv

In [3]:
with open('../data/neurons_david_data_with_spike_train.csv', 'r') as csv_f:
    reader = csv.reader(csv_f)
    csv_content = np.array(list(reader))
file_names = np.array([fn[0] for fn in csv_content])

In [9]:
np.where(file_names ==        '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2010_Adapt/Buckley_12deg/B101308/B101308_1342_List.smr',)

(array([79]),)

In [4]:
file_names

array(['/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O22/O22_2Adapt_BW5L_FW5LCFRDone.smr',
       '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O22/O22_2PreAdaptCFRDone.smr',
       '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O13/O13_3_adapt40pct.smr',
       '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O13/O13_3_adapt30pct.smr',
       '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O13/O13_3_preadapt.smr',
       '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O13/O13_3_adapt40pctB.smr',
       '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O41/O41_1_FW5L_BW5R_A.smr',
       '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O41/O41_1_pre.smr',
       '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O29/O29_4Adapt_FW5L_BW5RCFR611sec.smr',
       '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O29/O29_4PreAdaptCFRDone.smr',
       '/mnt/papers/Herzfeld_Nat_Ne

In [10]:
for ff in file_names[79:]:
    try:
        f_name = find_file(os.path.basename(ff), '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/')
        f_name_tacdec = find_file(os.path.basename(ff)+'.pkl', '../data/david_neurons')
        print(f_name)
        print(f_name_tacdec)
        neo_reader = Spike2IO(filename=f_name)
        neo_data = neo_reader.read()
        data_block = neo_data[0]
        seg = data_block.segments[0]

            # Load eye data
        HE = seg.analogsignals[0].as_array()
        t_HE = seg.analogsignals[0].times;

        VE = seg.analogsignals[1].as_array()
        t_VE = seg.analogsignals[1].times

        HT = seg.analogsignals[2].as_array()
        t_HT = seg.analogsignals[2].times

        VT = seg.analogsignals[3].as_array()
        t_VT = seg.analogsignals[3].times

        fs = seg.analogsignals[0].sampling_rate
        dt = seg.analogsignals[0].sampling_period

        t_start = 0
        t_end = 20000
        mysess = session(HT[np.where(np.logical_and((t_HT > t_start), (t_HT <= t_end)))], t_HT[np.where(np.logical_and((t_HT > t_start), (t_HT <= t_end)))],
                         VT[np.where(np.logical_and((t_VT > t_start), (t_VT <= t_end)))], t_VT[np.where(np.logical_and((t_VT > t_start), (t_VT <= t_end)))],
                         HE[np.where(np.logical_and((t_HE > t_start), (t_HE <= t_end)))], t_HE[np.where(np.logical_and((t_HE > t_start), (t_HE <= t_end)))],
                         VE[np.where(np.logical_and((t_VE > t_start), (t_VE <= t_end)))], t_VE[np.where(np.logical_and((t_VE > t_start), (t_VE <= t_end)))], fs, dt)
        mysess._cut_to_min_size()
        mysess._calc_target_velocity()
        mysess._calc_saccade_velocity()
        mysess._detect_target_jumps()
        mysess._detect_saccades()
        mysess._calc_error_vectors()

        # saving mysess
        output_fn = f_name_tacdec + '.session.pkl'
        print('writing')
        print(output_fn)
        with open(output_fn, 'wb') as output:
            print('writing {} ...'.format(output_fn))
            pickle.dump(mysess, output, pickle.HIGHEST_PROTOCOL)

    except Exception as e: 
        print(e)
        pass
        
    

/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2010_Adapt/Buckley_12deg/B101308/B101308_1342_List.smr
../data/david_neurons/B101308/B101308_1342_List.smr.pkl
writing
../data/david_neurons/B101308/B101308_1342_List.smr.pkl.session.pkl
writing ../data/david_neurons/B101308/B101308_1342_List.smr.pkl.session.pkl ...
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2010_Adapt/Buckley_12deg/B101308/B101308_1351_Adapt.smr
../data/david_neurons/B101308/B101308_1351_Adapt.smr.pkl
writing
../data/david_neurons/B101308/B101308_1351_Adapt.smr.pkl.session.pkl
writing ../data/david_neurons/B101308/B101308_1351_Adapt.smr.pkl.session.pkl ...
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2010_Adapt/Buckley_12deg/B101308/B101308_1443_AdaptForward.smr
../data/david_neurons/B101308/B101308_1443_AdaptForward.smr.pkl
writing
../data/david_neurons/B101308/B101308_1443_AdaptForward.smr.pkl.session.pkl
writing ../data/david_neurons/B101308/B101308_1443_AdaptForward.smr.pkl.session.pkl ...
/mnt/papers/Herzfel